In [1]:
import pandas as pd
import matplotlib.pyplot as plt

social = pd.read_csv("Aggregated_reddit_twitter.csv")
social = social.rename(columns={"date": "timestamp"})
social["timestamp"] = pd.to_datetime(social["timestamp"])



/tmp/ipykernel_65588/353302006.py:4: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  social = pd.read_csv("Aggregated_reddit_twitter.csv")


In [2]:
social = social[social.content != "u/circular360"]
social = social[social.content != "There you go"]
social = social[
    social.content != "Why force the other player to stay around? Just give the winning player the option to continue the game with a perma passing AI controlling the surrendering player."]

In [65]:
# crypto = pd.read_csv("transformed_crypto.csv")
# crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)
crypto = pd.read_csv("crypto_differences.csv")
crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)


In [66]:
# crypto.dtypes, social.dtypes

In [67]:
social = social.sort_values(by="timestamp")

In [68]:
# social.dtypes
sr = social.resample("T", on="timestamp").size()
freq = sr.reset_index()

freq = freq.rename(columns={0: "n_comments"})

In [69]:
mg = pd.merge(crypto, freq, on="timestamp", how="inner")

In [70]:
freq.idxmax()

timestamp     1126683
n_comments      67103
dtype: int64

In [71]:
t = freq.iloc[67103].timestamp
freq.iloc[67103].timestamp

Timestamp('2021-04-16 14:23:00')

In [72]:
import datetime

social.loc[(social['timestamp'] > t) & (social['timestamp'] < t + datetime.timedelta(minutes=2))]

,Unnamed: 0,id,title,author,timestamp,topics,likes,replies,link,content,other_scores,social
2632177,2632177,guq6zj5,NaN,SlappyMcLovin,2021-04-16 14:23:01,5555555,3.0,0.0,t3_mrr6wp,100 baht isn’t even worth 1 Doge,NaN,reddit
2632178,2632178,guq6zjo,NaN,mcathca,2021-04-16 14:23:01,Crypto_com,3.0,0.0,t3_ms453z,Thank you I’ll look into that when the app is ...,NaN,reddit
320222,320222,ms47x0,Dogefolder team/Folders@Home,staticchmbr,2021-04-16 14:23:01,dogecoin,3.0,7.0,https://www.reddit.com/r/dogecoin/comments/ms4...,\*\*EDIT (Sorry - Folding@Home) & I'm not JUS...,NaN,reddit
2632179,2632179,guq6zlf,NaN,Jeep2021,2021-04-16 14:23:02,dogecoin,1.0,0.0,t3_ms46pq,Someone needs to sue the hell out of them,NaN,reddit
2632180,2632180,guq6zn0,NaN,DarthLysergis,2021-04-16 14:23:02,Stellar,18.0,0.0,t3_ms3mv4,"I would like to add,\n\nThis truthfully has ev...",NaN,reddit
...,...,...,...,...,...,...,...,...,...,...,...,...
2632682,2632682,guq797b,NaN,mikenard77,2021-04-16 14:24:58,XRP,1.0,0.0,t3_ms1g1d,Follow the above steps without a PC first.,NaN,reddit
2632683,2632683,guq797f,NaN,zerotheelder,2021-04-16 14:24:58,CryptoCurrency,-1.0,0.0,t3_ms1man,"Not everyone can became billionaire, even the ...",NaN,reddit
2632684,2632684,guq797i,NaN,Loommy92,2021-04-16 14:24:58,MonsterHunter,-20.0,0.0,t3_mrznqs,Bruh what kinda BS rope is that? I swear handi...,NaN,reddit
2632685,2632685,guq797l,NaN,Reasonable_Cup6127,2021-04-16 14:24:58,dogecoin,1.0,0.0,t3_ms47kl,Yea buy in now but plan on holding for the lon...,NaN,reddit


In [73]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt


In [74]:
mg.shape, crypto.shape

((790319, 9), (875267, 8))

In [75]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset


In [123]:
shortmg = mg.loc[0:5000]
shortmg.dropna()

,Unnamed: 0,timestamp,XRP-PERP,SOL-PERP,ETH-PERP,DOGE-PERP,BTC-PERP,APE-PERP,n_comments


In [124]:
LAG = 30


class TimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)


class MyTimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data.loc[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)

# data = MyTimeSeriesDataset(X_data=shortmg[['n_comments', 'XRP-PERP', 'ETH-PERP']], y_data=shortmg['BTC-PERP'])

In [125]:
# Load your dataset into a pandas DataFrame called 'data'
from sklearn.preprocessing import MinMaxScaler

# Scale the features using MinMaxScaler

scaled_data = shortmg[['XRP-PERP', 'ETH-PERP', 'BTC-PERP', 'SOL-PERP', 'DOGE-PERP', 'n_comments']].to_numpy()

scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(scaled_data)
#
# Create sequences and targets for the RNN model
seq_length = 100
X_data = []
y_data = []

for i in range(len(scaled_data) - seq_length):
    X_data.append(scaled_data[i:i + seq_length])
    y_data.append(scaled_data[i + seq_length])

y_data = np.delete(y_data, -1, axis=1)
# print(y_data[1])
X_data, y_data = np.array(X_data), np.array(y_data)

In [126]:
train_size = int(len(X_data) * 0.8)
X_train, X_test = X_data[:train_size], X_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

In [127]:
from torch.utils.data import DataLoader

batch_size = 64
train_dataset = TimeSeriesDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = TimeSeriesDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [128]:
y_data.shape

(4901, 5)

In [129]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, output_size):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(input_size, d_model)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        x = self.fc(x[:, -1, :])
        return x

In [138]:
from torch import optim

input_size = X_data.shape[2]
hidden_size = 128
num_layers = 2
output_size = y_data.shape[1]

# model = RNNModel(input_size, hidden_size, num_layers, output_size)
model = TransformerModel(input_size, d_model=32, nhead=2, num_layers=2, output_size=output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [139]:
# from sgbd import SGBD
# optimizer = SGBD(model.parameters(), n_params=sum(p.numel() for p in model.parameters()), device=torch.device("cpu"),
#                          defaults={}, corrected=True, extreme=False,
#                          ensemble=None, step_size=None,
#                          thermolize_epoch=0, epochs=100, batch_n=len(train_loader))

In [140]:
import platform

num_epochs = 30

if "Linux" in platform.platform():
    model = torch.compile(model)

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0 and True:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(X_batch), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))

    model.eval()
    with torch.no_grad():
        test_loss = 0
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            test_loss += loss.item()

        test_loss = test_loss / len(test_loader)

        print(f"Epoch: {epoch + 1}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}")



[2023-04-27 10:16:11,720] torch._inductor.utils: [WARNING] using triton random, expect difference from eager

KeyboardInterrupt



In [110]:
y_data.var()

0.01577248895782091

In [61]:
crypto.head()

,Unnamed: 0,timestamp,XRP-PERP,SOL-PERP,ETH-PERP,DOGE-PERP,BTC-PERP,APE-PERP
0,0,2021-01-01 00:01:00,-1.582061,-1.194479,-1.932957,-1.453424,-1.070291,NaN
1,1,2021-01-01 00:02:00,-1.578833,-1.194382,-1.930775,-1.453357,-1.069996,NaN
2,2,2021-01-01 00:03:00,-1.579396,-1.194401,-1.931715,-1.453379,-1.067950,NaN
3,3,2021-01-01 00:04:00,-1.579809,-1.194401,-1.931568,-1.453377,-1.067929,NaN
4,4,2021-01-01 00:05:00,-1.578795,-1.194421,-1.932184,-1.453370,-1.069194,NaN
